In [117]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv


In [148]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = list(set(stopwords.words('english')))
stop_words.extend(['CA','ca','!','@','#','$','%','&','*','(',')',',','.','?','/','{','}','[',']'])

In [119]:
train = pd.read_csv("../input/nlp-getting-started/train.csv")
test = pd.read_csv("../input/nlp-getting-started/test.csv")

In [120]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [121]:
train['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [122]:
train['text'][2]

"All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"

In [123]:
train[train.keyword.notna()]['keyword'].values
# for x in train[train.keyword.notna()]['keyword'].values:
#     print(x.replace('%20',' '))

array(['ablaze', 'ablaze', 'ablaze', ..., 'wrecked', 'wrecked', 'wrecked'],
      dtype=object)

In [124]:
train['keyword'].isna().sum()

61

In [125]:
kws= list(train[train.keyword.notna()]['keyword'].values)

for x,y in zip(train.loc[train.keyword.isna(),'text'].head(10),train.loc[train.keyword.isna(),'keyword']):
#     for y in kws:
#         if y in x:
#             print(y)
#             train[train.text==x]['keyword'] = y
    print(x,y)    
    

Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all nan
Forest fire near La Ronge Sask. Canada nan
All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected nan
13,000 people receive #wildfires evacuation orders in California  nan
Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school  nan
#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires nan
#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas nan
I'm on top of the hill and I can see a fire in the woods... nan
There's an emergency evacuation happening now in the building across the street nan
I'm afraid that the tornado is coming to our area... nan


In [126]:
for x in train.loc[train.keyword.isna(),'text']:
    for y in kws:
        if y.lower() in x.lower():
            print(y)
            train.loc[train.text==x,'keyword'] = y
            break

earthquake
fire
evacuation
evacuation
fire
fire
disaster
fire
emergency
tornado
flood
flood
flood
crash
army
explode
flood
seismic
siren
attack
earthquake
cyclone
bomb
explode
flood
storm
debris
collide
earthquake
evacuation
siren
quarantine
evacuate
bomb
bomb
storm
collapse
fire
volcano
collide
fire


In [127]:
train['keyword'].isna().sum()

20

In [128]:
for x in train.loc[train.keyword.isna(),'text']:
    print(x.lower())

three people died from the heat wave so far
what's up man?
i love fruits
summer is lovely
my car is so fast
what a goooooooaaaaaal!!!!!!
this is ridiculous....
london is cool ;)
love skiing
what a wonderful day!
looooool
no way...i can't eat that shit
was in nyc last week!
love my girlfriend
cooool :)
do you like pasta?
the end!
heat wave warning aa? ayyo dei. just when i plan to visit friends after a year.
i just heard a really loud bang and everyone is asleep great
#breaking #la refugio oil spill may have been costlier bigger than projected http://t.co/5uecmcv2pk


Now that we've filled the keywords of most of disaster tweets,the others don't really matter

Still we've countable number(3) of disaster tweets left without keywords of which we can manually fill 2 of them with heat wave ;)

In [129]:
for x in train.loc[train.keyword.isna(),'text']:
    if 'heat wave' in x.lower():
        train.loc[train.text==x,'keyword'] = 'heat wave'
    elif 'oil spill' in x.lower():
        train.loc[train.text==x,'keyword'] = 'oil spill'
        

In [130]:
train.keyword.isna().sum()

17

In [131]:
for x in train.loc[train.keyword.isna(),'text']:
    print(x.lower(),train.loc[train.text==x,'target'])

what's up man? 15    0
Name: target, dtype: int64
i love fruits 16    0
Name: target, dtype: int64
summer is lovely 17    0
Name: target, dtype: int64
my car is so fast 18    0
Name: target, dtype: int64
what a goooooooaaaaaal!!!!!! 19    0
Name: target, dtype: int64
this is ridiculous.... 20    0
Name: target, dtype: int64
london is cool ;) 21    0
Name: target, dtype: int64
love skiing 22    0
Name: target, dtype: int64
what a wonderful day! 23    0
Name: target, dtype: int64
looooool 24    0
Name: target, dtype: int64
no way...i can't eat that shit 25    0
Name: target, dtype: int64
was in nyc last week! 26    0
Name: target, dtype: int64
love my girlfriend 27    0
Name: target, dtype: int64
cooool :) 28    0
Name: target, dtype: int64
do you like pasta? 29    0
Name: target, dtype: int64
the end! 30    0
Name: target, dtype: int64
i just heard a really loud bang and everyone is asleep great 7593    0
Name: target, dtype: int64


In [132]:
train['keyword'] = train['keyword'].fillna('general')

In [133]:
train['keyword'].isna().sum()

0

Yeah ! We've filled NaN values with appropraite values in keywords

In [134]:
train.head()

,id,keyword,location,text,target
0,1,earthquake,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,fire,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,evacuation,NaN,All residents asked to 'shelter in place' are ...,1
3,6,evacuation,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,fire,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [135]:
train.loc[train['location'].isna(),'text']

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 2533, dtype: object

In [136]:
train['location'].isna().sum()

2533

In [137]:
train.shape

(7613, 5)

So around 5000 locations are identified. We'll use them to try fill the unknown locations

Since we've extracted the keywords,let's remove the #,@ from text so that few locations in #,@ can be extracted

In [139]:
train['text'][0][0]

'O'

In [152]:
locs = list(train.loc[train.location.notna(),'location'].values)


for x in train.loc[train['location'].isna(),'text']:
    for y in locs:
        if y.lower() in word_tokenize(x.lower()) and y.lower() not in stop_words:
            train.loc[train['text']==x,'location'] = y
            
            break
        
    

In [153]:
train.location.isna().sum()

1990

In [154]:
train.to_csv('modified1.csv')

Obviously, the above method wasn't accurate as many locations weren't appropriate. But most of the irrelevant entries we've made are for non disaster tweets while disastrous tweets got proper location entries. So,it won't degrade our model performance

In [155]:
train.loc[train['location'].isna(),'text']

2       All residents asked to 'shelter in place' are ...
7       I'm on top of the hill and I can see a fire in...
8       There's an emergency evacuation happening now ...
9       I'm afraid that the tornado is coming to our a...
10            Three people died from the heat wave so far
                              ...                        
7600    Evacuation order lifted for town of Roosevelt:...
7602    a siren just went off and it wasn't the Forney...
7604    #WorldNews Fallen powerlines on G:link tram: U...
7605    on the flip side I'm at Walmart and there is a...
7608    Two giant cranes holding a bridge collapse int...
Name: text, Length: 1990, dtype: object

In [156]:
train['location'] = train['location'].fillna('unknown')

In [157]:
train.location.isna().any()

False

In [158]:
train.head()

,id,keyword,location,text,target
0,1,earthquake,US,Our Deeds are the Reason of this #earthquake M...,1
1,4,fire,Canada,Forest fire near La Ronge Sask. Canada,1
2,5,evacuation,unknown,All residents asked to 'shelter in place' are ...,1
3,6,evacuation,California,"13,000 people receive #wildfires evacuation or...",1
4,7,fire,Alaska,Just got sent this photo from Ruby #Alaska as ...,1


In [159]:
ids = train['id']
train.drop(['id'],axis=1,inplace=True)

In [160]:
train.head()

,keyword,location,text,target
0,earthquake,US,Our Deeds are the Reason of this #earthquake M...,1
1,fire,Canada,Forest fire near La Ronge Sask. Canada,1
2,evacuation,unknown,All residents asked to 'shelter in place' are ...,1
3,evacuation,California,"13,000 people receive #wildfires evacuation or...",1
4,fire,Alaska,Just got sent this photo from Ruby #Alaska as ...,1


In [162]:
target = train['target']
train.drop(['target'],axis=1,inplace=True)

In [163]:
train.head()

,keyword,location,text
0,earthquake,US,Our Deeds are the Reason of this #earthquake M...
1,fire,Canada,Forest fire near La Ronge Sask. Canada
2,evacuation,unknown,All residents asked to 'shelter in place' are ...
3,evacuation,California,"13,000 people receive #wildfires evacuation or..."
4,fire,Alaska,Just got sent this photo from Ruby #Alaska as ...
